In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

## 패키지 설치 및 Drive 연결

In [2]:
!pip install PyDrive
# pip install torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torchvision --upgrade --index-url https://download.pytorch.org/whl/cu118
# !pip install --upgrade torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python
!pip install opencv-python-headless
!pip install torchvision
!pip install google-api-core==2.11.0 google-api-python-client==2.79.0 google-auth==2.16.1 google-auth-httplib2==0.1.0 google-auth-oauthlib==0.4.1 googleapis-common-protos==1.58.0
!pip install --upgrade google-auth-oauthlib
!pip install requests google-auth-oauthlib google-auth-httplib2
# !pip install --force-reinstall sympy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.1%2Bcu118-cp310-cp310-linux_x86_64.whl (857.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp310-cp310-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached 

In [3]:
# YOLOv8 모델  설치
!pip3 install tqdm
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
# 라이브러리 불러오기
import os
import time
import pickle


In [66]:
## 토큰이 만료되었을 경우!!!
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools

obj = lambda: None

# OAuth 2.0 설정을 위한 딕셔너리
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# OAuth 2.0 인증을 위한 설정
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secrets.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)


In [ ]:
# Google Drive API를 사용하여 인증 관리
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# Google OAuth 인증 절차
gauth = GoogleAuth()


# 이미 저장된 자격 증명 파일이 있으면 로드
if os.path.exists("credentials.json"):
    gauth.LoadCredentialsFile("credentials.json")
    if gauth.access_token_expired:  # 토큰이 만료되었으면 갱신
        print("토큰 만료로 갱신")
        gauth.Refresh()
    else:  # 유효한 토큰이 있으면 인증 절차 건너뜀
        print("토큰이 있어 패스")
        gauth.Authorize()
else:
    # 자격 증명 파일이 없으면 새로 인증
    gauth.CommandLineAuth() # 이 과정을 통해 브라우저가 열리고, Google 계정 인증이 이루어짐
    gauth.SaveCredentialsFile("credentials.json")  # 자격 증명 파일 저장



In [7]:
# Google Drive 객체 생성
drive = GoogleDrive(gauth)

## Drive에서 파일을 불러와 딕셔너리에 저장

### 하위 폴더까지 탐색하는 코드

In [9]:
# 재귀적으로 폴더 탐색하여 파일 목록 가져오기
def recursive_list_files(drive, folder_id):
    files = []
    print(f"폴더 탐색 중: {folder_id}")  # 폴더 ID 출력
    # 현재 폴더 내 파일 및 폴더 목록 가져오기
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 폴더인 경우 재귀 호출
            files.extend(recursive_list_files(drive, file['id']))
        else:
            files.append(file)
    return files

### Drive API 호출 시 에러가 발생하면 일정 시간 대기 후 재시도

In [23]:
from googleapiclient.errors import HttpError

# Google Drive API 호출 시 에러가 발생하면 일정 시간 대기 후 재시도
def drive_list_files_with_retry(drive, folder_id, retries=10, wait_time=5):
    try:
        return drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false", 'maxResults': 400}).GetList()
    except HttpError as error:
        if retries > 0:
            print(f"HttpError 발생: {error}. {wait_time}초 후 재시도합니다... 남은 재시도 횟수: {retries}")
            time.sleep(wait_time)
            return drive_list_files_with_retry(drive, folder_id, retries-1, wait_time*1.5 )  # 대기 시간 증가
        else:
            print(f"HttpError 발생: {error}. 재시도 횟수 초과.")
            raise error

# 캐시 파일 경로 설정
label_cache_file = 'lane_label_files.pkl'

# 캐시 파일에서 데이터 불러오기
def load_cached_files(cache_file):
    if os.path.exists(cache_file):
        print('캐시 파일에서 데이터 불러오기...')
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

# 캐시 파일 저장하기
def save_cached_files(files, cache_file):
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())  # 현재 시간 포맷

    with open(cache_file, 'wb') as f:
        pickle.dump(files, f)
        print(f"[{current_time}] 캐시 저장 완료. 현재까지 탐색된 파일 수: {len(files)}")


# 재귀적으로 폴더 탐색하여 파일 목록 가져오기
def recursive_list_files(drive, folder_id, cache_file, files=None, save_interval=500):
    if files is None:
        files = load_cached_files(cache_file)
        
    print(f"폴더 탐색 중: {folder_id}")  # 폴더 ID 출력
    
    # 현재 폴더 내 파일 및 폴더 목록 가져오기
    file_list = drive_list_files_with_retry(drive, folder_id)
    # file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # 폴더인 경우 재귀 호출
            files = recursive_list_files(drive, file['id'],cache_file, files, save_interval)
        else:
            files.append(file)

        # 일정 간격(save_interval)마다 캐시 파일로 저장
        if len(files) % save_interval == 0:
            save_cached_files(files,cache_file)

    return files

### 라벨 데이터 캐싱 + 딕셔너리에 저장하는 코드

In [ ]:
# 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


start_time = time.time()

# 라벨 및 원천 데이터의 최상위 폴더 ID
training_folder_id = '15KO-jwlD4JFxWO2vcODvl89qg_qmgHIp'


# 캐시 파일 경로 설정
label_cache_file = 'lane_label_files.pkl'



# Google Drive에서 가져온 폴더 ID 예시
label_folder_id = '15aPJnLjR2vJ3okuPT5Vl-Ynf41M7Dikn'  # 라벨링 데이터 폴더 ID를 수동으로 설정 

# # 캐시 파일이 있는지 확인하고 불러오기
# if os.path.exists(label_cache_file):
#     print('캐시 파일에서 라벨 데이터 불러오기...')
#     with open(label_cache_file, 'rb') as f:
#         label_files = pickle.load(f)
# else:
#     print('캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...')

# 캐시 파일이 있는지 확인하고 불러오기
label_files = load_cached_files(label_cache_file)

# if len(label_files) == 0:
#     print('캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...')

if label_folder_id:
    # 라벨 파일 가져오기
    print('라벨쪽 실행시작...')
    label_files = recursive_list_files(drive, label_folder_id, label_cache_file)
    print(f"라벨 파일 수: {len(label_files)}")

    # 파일 목록을 캐시 파일에 저장
    save_cached_files(label_files, label_cache_file)
else:
    print("라벨 폴더의 ID를 찾을 수 없습니다.")
    label_files = []


    # if label_folder_id:

    #     # 라벨 파일 가져오기
    #     print('라벨쪽 실행시작...')
    #     label_files = recursive_list_files(drive, label_folder_id, label_cache_file)
    #     print(f"라벨 파일 수: {len(label_files)}")



    #     # 파일 목록을 캐시 파일에 저장
    #     save_cached_files(label_files, label_cache_file)
    # else:
    #     print("라벨 폴더의 ID를 찾을 수 없습니다.")
    #     label_files = []

# 캐시된 데이터에서 딕셔너리 생성
label_dict = {file['title']: file['id'] for file in label_files if file['title'].endswith('.json')}


# 라벨 파일 목록 일부 출력
print("라벨 파일 목록 일부:", list(label_dict.items())[:5])

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


In [ ]:
print("라벨 파일 목록 일부:", list(label_dict.items())[60690:])
print(len(label_dict))

### 원천 데이터 캐싱 + 딕셔너리에 저장하는 코드

In [ ]:
# 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


start_time = time.time()

# 라벨 및 원천 데이터의 최상위 폴더 ID
training_folder_id = '15KO-jwlD4JFxWO2vcODvl89qg_qmgHIp'

# 캐시 파일 경로 설정
image_cache_file = 'lane_image_files.pkl'

# Google Drive에서 가져온 폴더 ID
iamge_folder_id = '16SWhdIvoJ7ExlXLMUZkZL1E0qxJHEMTj'  # 원천 데이터 폴더 ID를 수동으로 설정 


# 캐시 파일이 있는지 확인하고 불러오기
iamge_files = load_cached_files(iamge_folder_id)

if iamge_folder_id:
    # 원천 파일 가져오기
    print('원천쪽 실행시작...')
    iamge_files = recursive_list_files(drive, iamge_folder_id, iamge_cache_file)
    print(f"사진 파일 수: {len(iamge_files)}")

    # 파일 목록을 캐시 파일에 저장
    save_cached_files(iamge_files, iamge_cache_file)
else:
    print("이미지 폴더의 ID를 찾을 수 없습니다.")
    image_files = []


# 캐시된 데이터에서 딕셔너리 생성
image_dict = {file['title']: file['id'] for file in image_files if file['title'].endswith('.jpg')}

print("이미지 파일 목록 일부:", list(image_dict.items())[:5])

# 실행 종료 시간 기록
end_time = time.time()
# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"코드 실행 시간: {execution_time:.2f}초")


In [ ]:
# # 라벨과 원천 한 번에 처리하는 코드
# # 캐싱을 이용하여 가져온 파일 목록을 로컬로 저장하고, 저장된걸 불러오는 방식


# start_time = time.time()

# # 라벨 및 원천 데이터의 최상위 폴더 ID
# training_folder_id = '15KO-jwlD4JFxWO2vcODvl89qg_qmgHIp'


# # 캐시 파일 경로 설정
# label_cache_file = 'lane_label_files.pkl'
# image_cache_file = 'lane_image_files.pkl'


# # Google Drive에서 가져온 폴더 ID 예시
# label_folder_id = '15aPJnLjR2vJ3okuPT5Vl-Ynf41M7Dikn'  # 라벨링 데이터 폴더 ID를 수동으로 설정 
# source_folder_id = '16SWhdIvoJ7ExlXLMUZkZL1E0qxJHEMTj'  # 원천 데이터 폴더 ID를 수동으로 설정 

# # 캐시 파일이 있는지 확인하고 불러오기
# if os.path.exists(label_cache_file) and os.path.exists(image_cache_file):
#     print('캐시 파일에서 데이터 불러오기...')
#     with open(label_cache_file, 'rb') as f:
#         label_files = pickle.load(f)
#     with open(image_cache_file, 'rb') as f:
#         image_files = pickle.load(f)
# else:
#     print('캐시 파일이 없습니다. Google Drive에서 파일 목록 가져오기...')


#     if label_folder_id and source_folder_id:

#         # 라벨 파일 가져오기
#         print('라벨쪽 실행시작...')
#         label_files = recursive_list_files(drive, label_folder_id)
#         print(f"라벨 파일 수: {len(label_files)}")

#         # 원천 파일 가져오기
#         print('원천 데이터 가져오기 시작...')
#         image_files = recursive_list_files(drive, source_folder_id)
#         print(f"이미지 파일 수: {len(image_files)}")

#         # label_files = []
#         # image_files = []

#         # # 라벨 데이터 가져오기
#         # print('라벨쪽 실행시작')
#         # label_folders = ['A', 'B', 'C']
#         # violation_types = ['BLUE', 'SHOULDER', 'WHITE', 'YELLOW']
        
#         # for folder in label_folders:
#         #     for violation in violation_types:
#         #         folder_path = f'라벨링데이터/TL/{folder}/{violation}'
#         #         subfolders = drive.ListFile({'q': f"'{label_folder_id}' in parents and trashed=false"}).GetList()
#         #         for subfolder in subfolders:
#         #             files = drive.ListFile({'q': f"'{subfolder['id']}' in parents and trashed=false"}).GetList()
#         #             label_files.extend(files)
#         # print(f"라벨 파일 수: {len(label_files)}")  # 라벨 파일 개수 출력

#         # # 원천 데이터 가져오기
#         # print('원천 데이터 가져오기 시작...')
#         # ts_folders = ['TS1', 'TS2', 'TS3']  # TS1, TS2, TS3 폴더
#         # for ts in ts_folders:
#         #     folder_path = f'원천데이터/{ts}/A'
#         #     subfolders = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()
#         #     for subfolder in subfolders:
#         #         files = drive.ListFile({'q': f"'{subfolder['id']}' in parents and trashed=false"}).GetList()
#         #         image_files.extend(files)
#         # print(f"이미지 파일 수: {len(image_files)}")  # 이미지 파일 개수 출력

#         # 파일 목록을 캐시 파일에 저장
#         with open(label_cache_file, 'wb') as f:
#             pickle.dump(label_files, f)
#         with open(image_cache_file, 'wb') as f:
#             pickle.dump(image_files, f)
#     else:
#         print("라벨 또는 원천 폴더의 ID를 찾을 수 없습니다.")
#         label_files = []
#         image_files = []

# # 캐시된 데이터에서 딕셔너리 생성
# label_dict = {file['title']: file['id'] for file in label_files if file['title'].endswith('.json')}
# image_dict = {file['title']: file['id'] for file in image_files if file['title'].endswith('.jpg')}


# # 라벨 및 이미지 파일 목록 일부 출력
# print("라벨 파일 목록 일부:", list(label_dict.items())[:5])
# print("이미지 파일 목록 일부:", list(image_dict.items())[:5])

# # 실행 종료 시간 기록
# end_time = time.time()
# # 실행 시간 계산 및 출력
# execution_time = end_time - start_time
# print(f"코드 실행 시간: {execution_time:.2f}초")
